In [ ]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7
###MeCabを利用する際に必要な呪文

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-locale-perl libfcgi-perl
  libhtml-parser-perl libhtml-tagset-perl libhttp-date-perl
  libhttp-message-perl libio-html-perl libio-string

In [ ]:
###形態素解析

import MeCab

text = "私はお昼休みに美味しいカレーライスを食べました。"

m = MeCab.Tagger("-Ochasen")
m.parse("")

def mecab_sep(text):
    node = m.parseToNode(text)

    words_list=[]

    while node:
        if node.feature.split(",")[0]=="動詞":
            words_list.append(node.feature.split(",")[6])
        elif node.feature.split(",")[0]=="形容詞":
            words_list.append(node.feature.split(",")[6])
        else:
            words_list.append(node.surface)
        node = node.next


    return words_list[1:-1]

In [ ]:
mecab_sep("こんにちは、これからよろしくお願いします。")

['こんにちは', '、', 'これから', 'よろしく', 'お願い', 'する', 'ます', '。']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
#ベクトル化に必要
def calc_vecs(docs):
    vectorizer = TfidfVectorizer(analyzer=mecab_sep)
    vecs=vectorizer.fit_transform(docs)
    return vecs.toarray()
#calc_vecs(docs)はベクトル化を行う関数として定義


import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
#コサイン類似度に必要




input_doc = "あなたは犬がすごく好きです。"
#診断したいユーザーの自然言語データを想定



target_docs_df = pd.read_csv("target_docs.csv",encoding='cp932')
#ファイルの読み込み(あらかじめ用意されているうつ病患者の自然言語データを全て繋げたデータの読み込みを想定)
target_docs = target_docs_df["文章リスト"].tolist()
#tolist()はリスト型つまり["文章１","文章２",・・・]という形にする
#target_docsはこれまでのうつ病患者の自然言語データを全て繋げたデータを想定



all_docs = [input_doc] + target_docs
all_docs_vecs = calc_vecs(all_docs)
#calc_vecs()を利用してベクトル化(上記にて定義した)
#all_docs_vecs=[診断したいユーザーの自然言語データのベクトル情報, これまでのうつ病患者の自然言語データを全て繋げたデータのベクトル情報]
#all_docs_vecs[0]=診断したいユーザーの自然言語データのベクトル情報
#all_docs_vecs[1]=これまでのうつ病患者の自然言語データを全て繋げたデータのベクトル情報

In [ ]:
similarity = cosine_similarity([all_docs_vecs[0]],[all_docs_vecs[1]])
#cos類似度を計算出来る関数
similarity
#cos近似値の結果(診断したいユーザーの自然言語データ VS これまでのうつ病患者の自然言語データを全て繋げたデータ)

array([[0.77348536]])